# Two Parameter Quantum Annealing With Multi-Body Interactions
### Author : David James Fulton


Presented by the author for the degree of MPhys, Physics, April 2017.
Last Udated : 17/04/2017

--------------------------------------------------------------------------------------------------------------------------------
##### The following material is in support for the report. It is comprised and is refered to in the appendix section A.3.
--------------------------------------------------------------------------------------------------------------------------------
In the cell following the inports, I present a simplified version of my anneal object, complete with function required for its methods.

In [ ]:
from __future__ import division
import numpy as np
import scipy.linalg
import matplotlib.pyplot as plt
%matplotlib notebook

In [ ]:

########################################### SIGMA MATRICES ###########################################################
#                              define the pauli matrices we are using
sigma_x = np.matrix('0. 1.; 1. 0.')
sigma_z = np.matrix('1. 0.; 0. -1.')

def sigma_xi(i,N):
    '''This returns the sigma_x operator appplied to the ith qubit in the Hilbert space of N qubits'''
    # note i is indexed from 0 here, not 1
    return np.kron(np.eye(2**i), np.kron(sigma_x,np.eye(2**(N-(i+1)))))

def sigma_zi(i,N):
    '''This returns the sigma_z operator appplied to the ith qubit in the Hilbert space of N qubits'''
    # note i is indexed from 0 here, not 1
    return np.kron(np.eye(2**i), np.kron(sigma_z,np.eye(2**(N-(i+1)))))
    



####################################### DIAGONALISATION FUNCTIONS ####################################################
#                        the Anneal class is dependent on the next two functions
def Diagonaliser(M):
    '''This function takes a hermitian matrix and returns the unitary transform matrices to change to and from the
    eigenbasis as well as the eigenvalues for the matrix.'''
    eigvals, eigvecs = sp.linalg.eigh(M)
    P = np.matrix(eigvecs)
    P_in = P.H
    return eigvals,P,P_in
    
def make_diag(d):
    '''this is a wrapper function to make a diagonal matrix given a series of values to go on the diagonal'''
    return np.matrix(np.diag(d))
    

def Projection(psi,v):
        '''Calculates the probability of measuring the wavestate in an eigenstate v'''
        Prob = abs(np.dot(psi.getH(),v)**2).item(0)
        return Prob
    
####################################### ANNEALING SCHEDULE FUNCTIONS #################################################

class Anneal():
    '''
    This object is to keep all the annealing stuff together in one neat package for each anneal completed.
    It applies can store a quite general prepared anneal and carry it out the run() method is used.
    This class is designed for closed system anneals only
    
    Each method explains itself but generally: 
        -the run function does the anneal
        -show_results shows the resuls of the anneal
    
    ~ The transverse field Hamiltonian H0 is concurrent with convention. See equation (2.1.6) int the report for its form.
    ~ The problem Hamiltonian is constructed by feeding in the parameters of the Ising formulation as in equation (2.1.6).
    
    ~ h are the problem hamiltonian fields input as an array.

    ~ J is a 2 dimensional array wich contains the Problem Hamiltonian coupling. Only the values above the diagonal of
      the J array are added to the coupling. This is consistant with equation (2.1.6).
    
    ~ 'tau' is the total time allowed for the annealing process.
    
    ~ 'points' is the resolution, the number of points used, in the integration.
    
    ~ 'A_schedules' is a list of transverse schedules functions, one for each qubit in the anneal.
    
    ~ 'qubits' is the number of qubits in the anneal. Equivalent to N_{log} in the report.
    
    
    '''
    
    def __init__(self,qubits,h,J,points,A_schedules,B_schedule,tau):
        '''This function sets up the attributes of the class'''
        
        # Store the basic information.
        self.qubits = qubits                               # N_q
        self.points = points                               # Number of points in integration
        self.tau = tau                                     # Total anneal time
        self.h = h                                         # Problem Hamiltonian fields
        self.J = J                                         # Problem Hamiltonian coupling
        self.H0, self.Hp = self.Construct_Hamiltonians()   # Sets up Hp and H0
        
        # Pick the values of s at which to integrate
        self.s = np.linspace(0,1,points)
        
        # Compute the schedule co-ordinates throughout the anneal
        self.A_i = np.zeros((points,qubits))
        for i in range(qubits):
                self.A_i[:,i] = A_schedules[i](points)
        
        self.B = B_schedule(points)
        
        
    def Construct_Hamiltonians(self):
        '''Constructs the transverse and problem Hamiltonians from the parameters input. Constructed according to 
           equation (2.1.6). This function is called in the init'''
        
        #initiate the Hamiltonians
        H0 = []
        Hp = np.matrix(np.zeros((2**self.qubits,2**self.qubits)))
        
        # unpack these for clarity of code
        h = self.h
        J = self.J
        Nq = self.qubits
        
        # loop through adding pieces to construct the hamiltonians 
        for i in range(Nq):
            # Store the individual pieces of the transverse field
            H0.append(sigma_xi(i,Nq))
            
            # Add on piecewise the local fields for the problem Hamiltonian
            Hp += h[i] * sigma_zi(i,Nq)
            
            # Add on piecewise the coupling between qubits
            for j in range(i+1,Nq): 
                Hp += J[i][j] * sigma_zi(i,Nq) * sigma_zi(j,Nq)     
  
        return H0,Hp
    
    def H(self,n):
        '''Return the time dependent Hamiltonian at the integration point n'''
        # calculate -\sum_{i} -A_i(s_n)H_0
        Ha = sum([-self.A_i[:,i][n] * self.H0[i] for i in range(self.qubits)])
        
        # calculate B(s_n)H_P
        Hb = self.B[n] * self.Hp
        
        return Ha+Hb
    

    def euler_update(self,dt,n):
        '''The euler update function applied at each time step. This function completes the integration
        step and stores some important values.'''
        # Calculate the eigenvalues and eigenvectors of the Hamiltonian. 
        # Use them to create matrices to rotate in and out of the eigenbasis.
        d,P,P_in = Diagonaliser(self.H(n))
        
        # Store energy eigenvalues.
        self.eigenvals.append(d)
        
        # Calculate the new state by applying U_n |Psi_{n-1}>
        self.states.append(P*make_diag(np.exp(-1.j*d*dt))*P_in*self.states[-1])
        
        # find and store the probability of measuring the qubits to be in any of the degenerate problem ground states
        self.Prob.append(sum([Projection(self.states[-1],prob_x0_k) for prob_x0_k in self.problem_x0s]))
        
        
    def run(self):
        '''This function runs the annealing protocol and creates attributes to store the data within the object'''
        
        # find the ground state of the initial Hamiltonian, -H0
        eigenvals0, eigenvecs0,eigenvecs0_in = Diagonaliser(-sum(self.H0))
        self.Psi0 = eigenvecs0[:,eigenvals0.argmin()] * (1.+0.j) # this is the ground state
        
        # find all degenerate ground states of the problem Hamiltonian, Hp 
        eigenvalsp, eigenvecsp,eigenvecsp_in = Diagonaliser(a.Hp)
        self.problem_x0s = [eigenvecsp[:,k] for k in np.where(eigenvalsp==min(eigenvalsp))[0]] # these are the ground states
        
        # set the time step
        dt = self.tau/self.points
        
        # set up data stores for the anneal whilst setting the inital state to be th ground state on the base hamiltonian
        self.states = [self.Psi0] # store the vector wavestate
        self.eigenvals =[] # store the energy eigenvalues
        self.Prob = [] # store the probability of being in one of the ground states
        
        # carry out the anneal whilst storing the state, energy eigenvalues and problem ground state probability
        for n in range(self.points):
            self.euler_update(dt,n)
        


--------------------------------------------------------------------------------------------------------------------------------
###### We will next set up a single anneal for the CZW device using a simple schedule.
--------------------------------------------------------------------------------------------------------------------------------

Here $A_{i}(s)=A_0(s)=1-s$ and $B(s)=s$

In [ ]:
# Define number of logical qubits, and hence deduce the number of ancilla and total number of qubits
# This number defines the the number of bodies in the effective N-body interaction of the CZW device
log_qub = 3

anc_qub = log_qub
qubits = log_qub*2
points = 1000 # number of integration points
tau = 10 # time for the anneal to be carried out in. (in NU)


########### THE FOLLOWING SECTION DERVIVES THE PARAMTERS NEEDED FOR THE CZW DEVICE ##########
# I RECOMMEND THE USER DOES NOT CHANGE THESE UNLESS FAMILIAR WITH REF [14] IN THE REPORT

# define the coupling parameters needed
JN = 0.1
q0 = 50.
Ja = 100.
Jl = Ja
hl = -Ja+q0


# construct h
hl = np.asarray([-Ja+q0]*log_qub) # h for the logical qubits
ha = np.asarray([-Ja*(2*i-log_qub)+q0 for i in range(1,log_qub+1)]) # h for the ancilla qubits
h = np.concatenate((hl,ha)) # the full h

# constuct J
J_log_log = Jl*np.triu(np.ones((log_qub,log_qub)), k=1) # the J component for the logical qubit interactions
J_log_anc = Ja*np.ones((log_qub,log_qub)) # interactions between logical and ancilla bits
Jt = np.concatenate((J_log_log,J_log_anc),axis=1) # make top half of J array
Jb = np.zeros(Jt.shape) # make bottom half of J array
J = np.concatenate((Jt,Jb),axis=0) # the full J

########################## SET THE ANNEALING SCHEDULES FOR EACH QUBIT ###############################
A_schedules = [lambda points: np.linspace(1,0,points)]*qubits
B_schedule = lambda points: np.linspace(0,1,points)


a = Anneal(qubits,h,J,points,A_schedules,B_schedule, tau)

In [ ]:
# We set up the anneal in the line above; now we run it. Note that above the user could change the Ising model parameters - 
#   above in order to carry out a different annealing problem. The user could also change the schedules.
a.run()

--------------------------------------------------------------------------------------------------------------------------------
###### Next we move to plot some of the important data from this anneal
--------------------------------------------------------------------------------------------------------------------------------

In [ ]:
plt.figure('Figure 1',figsize=(10,5))
plt.plot(a.s, a.Prob)
plt.xlabel('$s$')
plt.ylabel('$P(s)$')
plt.title('Population of Problem Ground State for CZW Anneal')
plt.show()

Figure 1: The above plot shows the probability of being in the Ising problem Hamiltonian ground state throughout the annealing process.

In [ ]:
max_level = 16 # we will plot this many energy levels starting from the ground state and working upwards
E = np.asarray(a.eigenvals)
plt.figure('Figure 2',figsize=(10,10))

for n in range(max_level):
    plt.plot(a.s,E[:,n],label=n)

plt.legend(loc='best', title = 'nth excited level')
plt.xlabel('$s$')
plt.ylabel('$P(s)$')
plt.title('Population of Problem Ground State for CZW Anneal')
plt.show()

Figure 2 : Above shows the energy levels thoughout the anneal for some of the lowest energy levels at the start of the anneal.

In [ ]:
print r'''The probability that the qubit array is measured to be in the ground state at the end of the annealing
process is : %s'''%a.Prob[-1]